# 2a: Compute Lorenz energy cycle

* This notebook assumes that `filter_data.ipynb` has been run with `lorenz = True`
* All terms computed in this notebook are depth-integrated, and are a function of **(time, y, x)**

Note: To get non-depth-integrated terms, remove all `.sum(dim='zi')` and `.sum(dim='zl')` statements.

In [1]:
filter_fac = 32
end_time = 2900 
extended_diags = False  # if true, compute some conversion terms in more than way (see below)

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import dask
from dask.diagnostics import ProgressBar

In [3]:
shape='Gaussian'
coarsen_fac=1
compl='simple'

In [4]:
run = 'nw2_0.03125deg_N15_baseline_hmix20'

## Get a view of Neverworld2 data

In [5]:
# static file with grid information
path = '/glade/p/univ/unyu0004/gmarques/NeverWorld2/baselines/'
st = xr.open_dataset('%s/%s/static.nc' % (path, run), decode_times=False)

In [6]:
ffile_pref = '/glade/scratch/noraloose/filtered_data/' 
chunks = {'time': 1, 'zl':1}
nr_days = 100

# filtered snapshots

sn_f_h = xr.open_zarr('%s/%s/snapshots_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_h' % (ffile_pref, run, end_time-nr_days+5, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
sn_f_u = xr.open_zarr('%s/%s/snapshots_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_u' % (ffile_pref, run, end_time-nr_days+5, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
sn_f_v = xr.open_zarr('%s/%s/snapshots_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_v' % (ffile_pref, run, end_time-nr_days+5, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
sn_f = xr.merge([sn_f_h, sn_f_u, sn_f_v])
    
# filtered averages
av_f_h = xr.open_zarr('%s/%s/averages_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_h' % (ffile_pref, run, end_time-nr_days+2, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
av_f_u = xr.open_zarr('%s/%s/averages_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_u' % (ffile_pref, run, end_time-nr_days+2, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
av_f_v = xr.open_zarr('%s/%s/averages_%08d_filtered_%s_coarse%i_fac%i_only_diffusion_%s_v' % (ffile_pref, run, end_time-nr_days+2, shape, coarsen_fac, filter_fac, compl), 
                           decode_times=False)
av_f = xr.merge([av_f_h, av_f_u, av_f_v])
    
str_filter = 'diff'

## Prepare NW2 grid information

In [7]:
Nx = np.size(st.xh)
Ny = np.size(st.yh)

In [8]:
from xgcm import Grid

coords = {'X': {'center': 'xh', 'outer': 'xq'},
            'Y': {'center': 'yh', 'outer': 'yq'},
            'Z': {'center': 'zl', 'outer': 'zi'} }
grid = Grid(st, coords=coords, periodic=['X'])

st['dxT'] = grid.interp(st.dxCu,'X')
st['dyT'] = grid.interp(st.dyCv,'Y')
st['dxBu'] = grid.interp(st.dxCv,'X')
st['dyBu'] = grid.interp(st.dyCu,'Y',boundary='fill')

metrics = {('X',):['dxCu','dxCv','dxT','dxBu'],
           ('Y',):['dyCu','dyCv','dyT','dyBu'],
           ('X', 'Y'): ['area_t', 'area_u', 'area_v']
          }

grid = Grid(st, coords=coords, periodic=['X'], metrics=metrics)

## New dataset for Lorenz cycle

In [9]:
ds = xr.Dataset() # new xarray dataset for terms in Lorenz cycle 

ds.attrs['filter_shape'] = 'Gaussian' 
ds.attrs['filter_factor'] = filter_fac

for dim in ['time','zl','yh','xh']:
    ds[dim] = av_f[dim]

# Energy reservoirs & their tendencies

### MPE & EPE

\begin{align}
 \text{PE} = \frac{1}{2}\sum_{n=0}^{N-1} g_n' \eta_n^2
 \qquad
 \text{MPE} = \frac{1}{2}\sum_{n=0}^{N-1} g_n' \bar{\eta}_n^2
 \qquad
 \text{EPE} = \overline{\text{PE}} - \text{MPE}
\end{align}
with $g_k' = g (\rho_{k+1} - \rho_k) / \rho_o$

In [10]:
rho_ref = 1000  # refernce density in NeverWorld2
# reduced gravity
gprime = 10 * grid.diff(av_f.zl,'Z',boundary='fill') / rho_ref
gprime[15] = np.nan

In [11]:
ds['MPE'] = (0.5 * gprime * av_f['eta']**2).sum(dim='zi')
ds['EPE'] = (0.5 * gprime * av_f['eta2']).sum(dim='zi') - ds['MPE']
ds['MPE'].attrs = {'units' : 'm3 s-2', 'long_name': 'Mean Potential Energy'}
ds['EPE'].attrs = {'units' : 'm3 s-2', 'long_name': 'Eddy Potential Energy'}

KeyError: 'eta'

### MPE & EPE tendencies
\begin{align} 
    \partial_t(\text{MPE}) = \sum_{n=0}^{N-1} g_n' \bar{\eta}_n \partial_t\bar{\eta}_n
     \qquad
     \partial_t(\text{EPE}) = \frac{1}{2}\sum_{n=0}^{N-1} g_n' \overline{\partial_t(\eta^2_n}) - \partial_t(\text{MPE}) 
\end{align}

In [31]:
ds['dMPEdt'] = (gprime * av_f['eta'] * av_f['deta_dt']).sum(dim='zi')
ds['dEPEdt'] = (0.5 * gprime * av_f['deta2_dt']).sum(dim='zi') - ds['dMPEdt']

KeyError: 'eta'

### MKE & EKE

\begin{align}
 \text{KE} = \frac{1}{2}\sum_{n=1}^{N} h_n (u_n^2 + v_n^2)
 \qquad
 \text{MKE} = \frac{1}{2}\sum_{n=1}^{N} \bar{h}_n (\bar{u}_n^2 + \bar{v}_n^2)
 \qquad
 \text{EKE} = \overline{\text{KE}} - \text{MKE}
\end{align}

In [18]:
MKE = 0.5 * av_f['h'] * (
    grid.interp((av_f['u']**2).fillna(value=0), 'X', metric_weighted=['X','Y'])  
    + grid.interp((av_f['v']**2).fillna(value=0), 'Y', metric_weighted=['X','Y']) 
)
ds['MKE'] = MKE.sum(dim='zl')

#av_f['hKE'] is filtered KE = 0.5 * h * (u^2 + v^2)
ds['EKE'] = av_f['hKE'].sum(dim='zl') - ds['MKE']  

ds['MKE'].attrs = {'units' : 'm3 s-2', 'long_name': 'Mean Kinetic Energy (non-TWA)'}
ds['EKE'].attrs = {'units' : 'm3 s-2', 'long_name': 'Eddy Kinetic Energy (non-TWA)'}

In [19]:
ds

<xarray.Dataset>
Dimensions:  (time: 20, xh: 1920, yh: 4480, zl: 15)
Coordinates:
  * time     (time) float64 2.802e+03 2.808e+03 ... 2.892e+03 2.898e+03
  * zl       (zl) float64 1.023e+03 1.023e+03 1.023e+03 ... 1.028e+03 1.028e+03
  * yh       (yh) float64 -69.98 -69.95 -69.92 -69.89 ... 69.92 69.95 69.98
  * xh       (xh) float64 0.01562 0.04688 0.07812 0.1094 ... 59.92 59.95 59.98
Data variables:
    MKE      (time, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    EKE      (time, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    dMKEdt   (time, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
    dEKEdt   (time, yh, xh) float64 dask.array<chunksize=(1, 4480, 1920), meta=np.ndarray>
Attributes:
    filter_shape:   Gaussian
    filter_factor:  32

### MKE & EKE tendencies
\begin{align} 
    \partial_t(\text{MKE}) =  \text{5-day-average}\left(\partial_t \text{MKE}\right) = \frac{1}{\tau_1-\tau_0} \int_{\tau_0}^{\tau_1} \partial_t (\text{MKE})\, dt = \frac{\text{MKE}(\tau_1) - \text{MKE}(\tau_0)}{\tau_1-\tau_0},
\end{align}
where the 5-day time interval is denoted by $[\tau_0, \tau_1]$. We can thus get the MKE tendencies from MKE snapshots. Similarly, we can compute

\begin{align} 
    \partial_t(\text{EKE}) =  \frac{\overline{\text{KE}}(\tau_1) - \overline{\text{KE}}(\tau_0)}{\tau_1-\tau_0} - \partial_t(\text{MKE})
\end{align}
from snapshots of `hKE` and the MKE snapshots.

In [28]:
if np.all(av_f.average_DT == av_f.average_DT[0]):
    deltat = av_f.average_DT[0] * 24 * 60 * 60
else: 
    raise AssertionError('averaging intervals vary')

In [34]:
# MKE tendency
MKE = 0.5 * sn_f['h'] * (
    grid.interp((sn_f['u']**2).fillna(value=0), 'X', metric_weighted=['X','Y'])  
    + grid.interp((sn_f['v']**2).fillna(value=0), 'Y', metric_weighted=['X','Y']) 
)

if np.array_equal(av_f.time_bnds[:,1], sn_f.time):
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        dMKEdt = (MKE - MKE.shift(time=1)) / deltat
        dMKEdt['time'] = av_f['h'].time
else: 
    raise AssertionError('av and sn datasets not compatitble')

dMKEdt = dMKEdt.where(av_f.time > av_f.time[0])  
dMKEdt = dMKEdt.chunk({'time': 1})
ds['dMKEdt'] = dMKEdt.sum(dim='zl')

ds['dMKEdt'].attrs = {'units' : 'm3 s-2', 'long_name': 'Mean Kinetic Energy tendency (non-TWA)'}

# EKE tendency
hKE = sn_f['hKE']
if np.array_equal(av_f.time_bnds[:,1], sn_f.time):
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        hKEdt = (hKE - hKE.shift(time=1)) / deltat
        hKEdt['time'] = av_f['h'].time
else: 
    raise AssertionError('av and sn datasets not compatitble')
 
hKEdt = hKEdt.where(av_f.time > av_f.time[0])  
hKEdt = hKEdt.chunk({'time': 1})
ds['dEKEdt'] = hKEdt.sum(dim='zl') - ds['dMKEdt']
ds['dEKEdt'].attrs = {'units' : 'm3 s-2', 'long_name': 'Eddy Kinetic Energy tendency (non-TWA)'}

## Energy conversion terms (cf. Figure 3a in Loose et al., 2022)

### EKE production $\Sigma^L$

\begin{align}
   \Sigma^L = \underbrace{- \sum_{n=1}^N\overline{h_n (u_n \partial_x M_n + v_n \partial_y M_n)}}_{\overline{\text{PE_to_KE+KE_BT}}} + \sum_{n=1}^N\bar{h}_n(\bar{u}_n \underbrace{\overline{\partial_x M_n}}_{-\overline{\text{PFu+u_BT_accel_visc_rem}}} + \bar{v}_n \underbrace{\overline{\partial_y M_n}}_{-\overline{\text{PFv+v_BT_accel_visc_rem}}} )
\end{align}

In [36]:
EKE_production = av_f['PE_to_KE+KE_BT'] - av_f['h'] / st['area_t']  * (
    grid.interp((av_f['u'] * (av_f['PFu+u_BT_accel_visc_rem']) * st['area_u']).fillna(value=0),'X')
    + grid.interp((av_f['v'] * (av_f['PFv+v_BT_accel_visc_rem']) * st['area_v']).fillna(value=0),'Y')
)
ds['EKE_production'] = EKE_production.sum(dim='zl')
ds['EKE_production'].attrs = {'units' : 'm3 s-3', 'long_name': 'EKE production (non-TWA)'}

### Baroclinic conversion $\Gamma^L$

With 
$$\mathcal{E} = \sum_{n=1}^N \left(\nabla\cdot(\overline{h_n\mathbf{u}_n}) \underbrace{- \overline{\nabla\cdot(h_n\mathbf{u}_n)}}_{\overline{\partial_t h_n}}\right) \bar{M}_n,
$$ 
we have

\begin{align}
 \Gamma^L & = -\left(\sum_{n=1}^N(\overline{h_n u_n}\cdot \partial_x \bar{M}_n + \overline{h_n v_n}\cdot \partial_y \bar{M}_n ) -\sum_{n=1}^N\bar{h}_n(\bar{u}_n \overline{\partial_x M_n} + \bar{v}_n \overline{\partial_y M_n})\right) - \mathcal{E}\\
 & =\Sigma^L
 \underbrace{-  \sum_{n=1}^N \left(\overline{M_n \left(\partial_x (h_n u_n) + \partial_y (h_n v_n)\right)} -\bar{M}_n\left(\overline{\partial_x (h_n u_n) + \partial_y (h_n v_n}\right)\right)}_\text{EPE tendency, see eqn (A5)}
 + \underbrace{ \sum_{n=1}^N \left(\overline{\partial_x(h_n u_n M_n) + \partial_y(h_n v_n M_n)} 
 - \left(\partial_x(\overline{h_n u_n} \bar{M}_n) + \partial_y (\overline{h_n v_n} \bar{M}_n)\right)\right)}_{-\mathcal{D},\text{ see eqn (A13)}}
 \end{align}

where the last identity follows from equations (A12) and (A13) in Loose et al. (2022). 

Thus, we have two options to compute $\Gamma^L$: via the first line, or via the second line. The two options would give the same result in a non-discretized world, but lead to small differences in our discretized world (in time and space). For Figure 8 in Loose et al. (2022), we computed $\Gamma^L$ via the second option, and this is the default below (`BC_conversion`). 

If you want to diagnose $\Gamma^L$ via the first option (`BC_conversion_alt`), set `extended_diags=True` at the top of this notebook. This option also assumes that you have run `filter_data.ipynb` with `lorenz=True` and `extended_diags=True`.

In [ ]:
# compute eddy pressure flux divergence, div = - D
uhM_mean = av_f['uh'] * grid.interp(av_f['MP'].fillna(value=0),'X', metric_weighted=['X','Y'])
uflux = grid.diff(uhM_mean.fillna(value=0),'X')
vhM_mean = av_f['vh'] * grid.interp(av_f['MP'].fillna(value=0),'Y', metric_weighted=['X','Y'], boundary='fill')
vflux = grid.diff(vhM_mean.fillna(value=0),'Y')
div_mean = (uflux + vflux).where(wet) / st.area_t  # finite volume discretization

# av_f[uhM_div] = filtered(div(uhM))
div = av_f['uhM_div'] - div_mean
div = div.chunk({'yh':Ny, 'xh':Nx})

ds['BC_conversion'] = ds['EKE_production'] + div.sum(dim='zl') + ds['dEPEdt']
ds['BC_conversion'].attrs = {'units' : 'm3 s-3', 'long_name': 'baroclinic conversion (non-TWA)'}

if extended_diags:
    # extra term E
    uflux = grid.diff(av_f['uh'].fillna(value=0), 'X')
    vflux = grid.diff(av_f['vh'].fillna(value=0), 'Y')
    div = (uflux + vflux) / st.area_t  # finite volume discretization
    extra_term = av_f['MP'] * (div + av_f['dhdt']) 

    conversion = (
        - grid.interp((av_f['uh'] / st['dyCu']) * (grid.derivative(av_f['MP'], 'X')), 'X')
        - grid.interp((av_f['vh'] / st['dxCv']) * (grid.derivative(av_f['MP'], 'Y', boundary='fill')), 'Y')
        - av_f['h'] / st['area_t'] * (
            grid.interp((av_f['u'] * (av_f['PFu+u_BT_accel_visc_rem']) * st['area_u']).fillna(value=0),'X')
            + grid.interp((av_f['v'] * (av_f['PFv+v_BT_accel_visc_rem']) * st['area_v']).fillna(value=0),'Y') 
        )
    )

    ds['BC_conversion_alt'] = (conversion + extra_term).sum(dim='zl') 
    ds['BC_conversion_alt'].attrs = {'units' : 'm3 s-3', 'long_name': 'baroclinic conversion (non-TWA), alternative computation'}

### MKE --> MPE
\begin{align}
   (\text{MKE} \to \text{MPE}) & = \underbrace{\sum_{n=1}^N\bar{h}_n \left(\bar{u}_n \overline{\partial_x M_n} + \bar{v}_n \overline{\partial_y M_n}\right)}_\text{see Figure 3a}\\
    & = \Gamma^A + \partial_t \text{MPE} +  \sum_n \nabla\cdot\left( \overline{h_n\mathbf{u}_n} \bar{M}_n\right),
\end{align}

where
$$
\Gamma^A =
-\left(\sum_{n=1}^N\overline{h_n\mathbf{u}_n}\cdot\nabla \bar{M}_n -\sum_{n=1}^N\bar{h}_n\bar{\mathbf{u}}_n\cdot\overline{\nabla M_n}\right) - \mathcal{E}
$$
and
$$
    \partial_t\text{MPE} = - \sum_n \nabla\cdot\left( \overline{h_n\mathbf{u}_n} \bar{M}_n\right) + \sum_n \overline{h_n \mathbf{u}_n} \cdot \nabla \bar{M}_n + \mathcal{E}
$$

Again, we have two options to compute $(\text{MKE} \to \text{MPE})$ via the first line, or via the second line. For Figure 8 in Loose et al. (2022), we chose the second option, and this is the default below (`MKE_to_MPE`). 

If you want to diagnose $(\text{MKE} \to \text{MPE})$ via the first option (`MKE_to_MPE_alt`), set `extended_diags=True` at the top of this notebook. This option also assumes that you have run `filter_data.ipynb` with `lorenz=True` and `extended_diags=True`.

In [37]:
uflux = grid.diff((av_f['uh'] * grid.interp(av_f['MP'].fillna(value=0), 'X', metric_weighted=['X','Y'])).fillna(value=0),'X')
vflux = grid.diff((av_f['vh'] * grid.interp(av_f['MP'].fillna(value=0),'Y',metric_weighted=['X','Y'],boundary='fill')).fillna(value=0),'Y')
div = (uflux + vflux) / st.area_t  # finite volume discretization
ds['MKE_to_MPE'] = ds['BC_conversion'] + ds['dMPEdt'] - div.sum(dim='zl')
ds['MKE_to_MPE'].attrs = {'units' : 'm3 s-3', 'long_name': 'MKE to MPE conversion (non-TWA)'}

if extended_diags:
    MKE_to_MPE_alt =  - av_f['h'] / st['area_t']  * (
        grid.interp((av_f['u'] * (av_f['PFu+u_BT_accel_visc_rem']) * st['area_u']).fillna(value=0),'X')
        + grid.interp((av_f['v'] * (av_f['PFv+v_BT_accel_visc_rem']) * st['area_v']).fillna(value=0),'Y')
    )
    ds['MKE_to_MPE_alt'] = MKE_to_MPE_alt.sum(dim='zl')
    ds['MKE_to_MPE_alt'].attrs = {'units' : 'm3 s-3', 'long_name': 'MKE to MPE conversion (non-TWA), alternative computation'}

KeyError: 'MP'

### EKE transport

\begin{align}
\mathcal{T}^L & =
\sum_{n=1}^N \left[
\underbrace{\overline{\nabla\cdot\left(
\mathbf{u}_n\frac{h_n|\mathbf{u}_n|^2}{2}\right)}}_\overline{\text{KE_adv}}
-\nabla\cdot\left(
\bar{\mathbf{u}}_n\underbrace{\frac{\bar{h}_n|\bar{\mathbf{u}}_n|^2}{2}}_\text{MKE}
\right)\right],
\end{align}

In [38]:
MKE_transport =  1 / st.area_t * (
        grid.diff((grid.interp(MKE.fillna(value=0),'X') * av_f['u'] * st.dyCu).fillna(value=0),'X')
        + grid.diff((grid.interp(MKE.fillna(value=0),'Y',boundary='fill') * av_f['v'] * st.dxCv).fillna(value=0),'Y')
)
MKE_transport = MKE_transport.chunk({'yh':Ny, 'xh':Nx})

EKE_transport = av_f['KE_adv'] - MKE_transport
ds['EKE_transport'] = EKE_transport.sum(dim='zl')
ds['EKE_transport'].attrs = {'units' : 'm3 s-3', 'long_name': 'EKE transport (non-TWA)'}

### Work done by eddy momentum fluxes

\begin{align}
(\text{ke}_u)_{Ij} & = 
-\overline{\texttt{u}}_{Ij}\cdot \texttt{area\_u}_{Ij}\cdot (\overline{\texttt{rvxv}}_{Ij} + \overline{\texttt{gKEu}}_{Ij})\\
(\text{ke}_v)_{iJ} & = -
\overline{\texttt{v}}_{iJ}\cdot \texttt{area\_v}_{iJ}\cdot (\overline{\texttt{rvxu}}_{iJ} + \overline{\texttt{gKEv}}_{iJ})
\end{align}

As residual:

 \begin{align*}
 (\text{KE Exchange})_{ij} 
   &=  \overline{\texttt{h}}_{ij}\frac{(\text{ke}_u)_{I-1j}+(\text{ke}_u)_{Ij}+(\text{ke}_v)_{iJ-1}+(\text{ke}_v)_{iJ}}{2\texttt{area\_T}_{ij}}\\
   &- \texttt{MKE}_{ij}/\overline{\texttt{h}_{ij}} \cdot \overline{\texttt{dhdt}_{ij}}\\
 &-  \frac{1}{2\texttt{area\_t}} \Big\{ \delta_i\Big[ \texttt{MKE}_{T\to U} \overline{\texttt{u}}\cdot\texttt{dyCu}\Big]
 +  \delta_j
   \Big[ \texttt{MKE}_{T\to V}\overline{\texttt{v}}\cdot\texttt{dxCv}\Big]
   \Big\}
\end{align*}




In [16]:
ke_u = - av_f['u'] * (av_f['CAu_visc_rem']) * st['area_u']
ke_v = - av_f['v'] * (av_f['CAv_visc_rem']) * st['area_v']

term1 = av_f['h'] * grid.interp(ke_u.fillna(value=0),'X') 
term1 = term1 + av_f['h'] * grid.interp(ke_v.fillna(value=0),'Y')  
term1 = term1/st['area_t']
 
# discretize like MKE
ufld = av_f['u']**2
tfld1 = grid.interp(ufld.fillna(value=0),'X') 
vfld = av_f['v']**2
tfld2 = grid.interp(vfld.fillna(value=0),'Y') 
term2 = - 0.5 * (tfld1 + tfld2) * av_f['dhdt']

MKE =  0.5 * av_f['h'] * (tfld1 + tfld2)
ufld2 = grid.interp(MKE.fillna(value=0),'X') * av_f['u'] * st.dyCu
vfld2 = grid.interp(MKE.fillna(value=0),'Y',boundary='fill') * av_f['v'] * st.dxCv
term3 = grid.diff(ufld2.fillna(value=0),'X')
term3 = term3 + grid.diff(vfld2.fillna(value=0),'Y')
term3 = - term3/st.area_t

data = term1 + term2 + term3
data = xr.where(st.wet,data,np.nan)
    
ds['Exchange_area'] = data.copy()
#ds['Exchange_area'].attrs = {'units' : ds['dEKEdt_sn_area'].units, 
#                                'long_name': 'KE Exchange (non-TWA)'}
ds['Exchange_area'] = ds['Exchange_area'].chunk({'yh':Ny, 'xh':Nx})



### Wind work on MKE & EKE reservoir

Wind work on MKE reservoir:
\begin{align}
    \sum_{n=1}^N \bar{h}_n (\bar{u}_n \overline{F^{u,\text{wind}}_n} + \bar{v}_n \overline{F^{v,\text{wind}}_n}), 
\end{align}

Wind work on MKE reservoir:
\begin{align}
    \sum_{n=1}^N \overline{\underbrace{h_n (u_n F^{u,\text{wind}}_n + v_n F^{v,\text{wind}}_n)}_\text{KE_stress}} 
    - \sum_{n=1}^N \bar{h}_n (\bar{u}_n \overline{F^{u,\text{wind}}_n} + \bar{v}_n \overline{F^{v,\text{wind}}_n}), 
\end{align}

In [18]:
MKE_wind_stress = av_f['h'] * (
    grid.interp((av_f['u'] * (av_f['du_dt_str_visc_rem'])).fillna(value=0), 'X', metric_weighted=['X','Y']) 
    + grid.interp((av_f['v'] * (av_f['dv_dt_str_visc_rem'])).fillna(value=0), 'Y', metric_weighted=['X','Y']) 
)
ds['MKE_wind_stress'] = MKE_wind_stress.sum(dim='zl')
ds['MKE_wind_stress'].attrs = {'units' : 'm3 s-3', 'long_name': 'Wind work on MKE reservoir (non-TWA)'}

ds['EKE_wind_stress'] = av_f['KE_stress'].sum(dim='zl') - ds['MKE_wind_stress']
ds['EKE_wind_stress'].attrs = {'units' : 'm3 s-3', 'long_name': 'Wind work on EKE reservoir (non-TWA)'}

### Horizontal friction on MKE & EKE reservoir

Horizontal friction on MKE reservoir:
\begin{align}
    \sum_{n=1}^N \bar{h}_n (\bar{u}_n \overline{F^{u,h}_n} + \bar{v}_n \overline{F^{v,h}_n}), 
\end{align}

Horizontal friction on MKE reservoir:
\begin{align}
    \sum_{n=1}^N \overline{\underbrace{h_n (u_n F^{u,h}_n + v_n F^{v,h}_n)}_\text{KE_horvisc}} 
    - \sum_{n=1}^N \bar{h}_n (\bar{u}_n \overline{F^{u,h}_n} + \bar{v}_n \overline{F^{v,h}_n}), 
\end{align}

In [ ]:
MKE_horizontal_viscosity = av_f['h'] * (
    grid.interp((av_f['u'] * (av_f['diffu_visc_rem'])).fillna(value=0), 'X', metric_weighted=['X','Y']) 
    + grid.interp((av_f['v'] * (av_f['diffv_visc_rem'])).fillna(value=0), 'Y', metric_weighted=['X','Y']) 
)
ds['MKE_horizontal_viscosity'] = MKE_horizontal_viscosity.sum(dim='zl')
ds['MKE_horizontal_viscosity'].attrs = {'units' : 'm3 s-3', 'long_name': 'Horizontal friction on MKE reservoir (non-TWA)'}

ds['EKE_horizontal_viscosity'] = av_f['KE_horvisc'].sum(dim='zl') - ds['MKE_horizontal_viscosity']
ds['EKE_horizontal_viscosity'].attrs = {'units' : 'm3 s-3', 'long_name': 'Horizontal friction on EKE reservoir (non-TWA)'}

In [ ]:
# MKE_vertical_stresses and EKE_vertical_stresses include contributions from bottom drag, vertical viscosity, and wind stress
MKE_vertical_stresses = av_f['h'] * (
    grid.interp((av_f['u'] * (av_f['du_dt_visc_rem'])).fillna(value=0), 'X', metric_weighted=['X','Y']) 
    + grid.interp((av_f['v'] * (av_f['dv_dt_visc_rem'])).fillna(value=0), 'Y', metric_weighted=['X','Y']) 
)
MKE_vertical_stresses = MKE_vertical_stresses.sum(dim='zl')
EKE_vertical_stresses = av_f['KE_visc'].sum(dim='zl') - MKE_vertical_stresses

# tease out bottom drag and vertical viscosity contribution
ds['MKE_bottom_drag'] = (
        MKE_vertical_stresses - ds['MKE_wind_stress']  # subtract wind stress contribution from vertical stresses
    ) * ds_mask['mask_bottom_and_below']  # extract the bit in the bottom layer
ds['MKE_vertical_viscosity'] = MKE_vertical_stresses - ds['MKE_wind_stress'] - ds['MKE_bottom_drag']

ds['EKE_bottom_drag'] = (
        EKE_vertical_stresses - ds['EKE_wind_stress']  # subtract wind stress contribution from vertical stresses
    ) * ds_mask['mask_bottom_and_below']  # extract the bit in the bottom layer
ds['EKE_vertical_viscosity'] = EKE_vertical_stresses - ds['EKE_wind_stress'] - ds['EKE_bottom_drag']


## Save EKE budget terms to netcdf

In [20]:
#ds = ds.transpose('time', 'zi', 'zl', 'yh', 'xh') # reorder coordinates

In [21]:
ds2 = xr.Dataset() 
fldlist = ['EKE_area', 'dEKEdt_sn_area','Transport_no_PG_area','Exchange_area',
           'Sources_area','Sinks_hor_area','Sinks_vert_area',
           'Transport_PG',
           'Sinks_vert_wind_area',
           'GM_term_area','dEPEdt','EPE','MPE','MPE2MKE_area',
           'MKE_sources_hor','MKE_sources_vert',
           'MKE_sources_vert_wind',
           'MKE_area']
for fld in fldlist:
    ds2[fld] = ds[fld]
ds2

,Array,Chunk
Bytes,20.64 GB,68.81 MB
Shape,"(4480, 1920, 20, 15)","(4480, 1920, 1, 1)"
Count,7805 Tasks,300 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,20.64 GB,68.81 MB
Shape,"(20, 15, 4480, 1920)","(1, 1, 4480, 1920)"
Count,10741 Tasks,300 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [ ]:
ffile = ffile_pref + run + '/EKE_%08d_%s_coarse%i_fac%i_%s' % (end_time-nr_days+2, 
                                                               shape, coarsen_fac, filter_fac, compl)
ffile

In [ ]:
ds2.to_zarr(ffile)